FastText
====

Nel notebook precedente siamo risuciti araggiungere una accuratezza nella fase di test dell  ~84% usando tutte le comuni tecniche di sentiment analisys. Qui andremo ad implementare un sistema con prestazioni comparabili ma significativamente più veloce e con metà parametri di trainin. Nello specifico andremo ad implementare il modello "FastText" descritto nel paper [Bag of Tricks for Efficient Text Classification](https://arxiv.org/abs/1607.01759).

Preparazione dei dati
----

Uno dei concetti chiavi del modello è il calcolo degli n-gram di una frase e aggiungerli alla fine della frase.
Qui andremo ad utilizzare i bigrammi (bi-grams). Brevemente i bigrammi sono delle coppie di parole/token che appaiono consecutivamente in una frase.

Per esempio nella frase "come va oggi ?", i bigrammi sono "come va, va oggi, oggi ?".

La funzione generate_bigrams prende in ingresso una frase già tokenizzata e calcola i bigrammi e li aggiunge alla fine della lista dei token.

In [1]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

qui un esempio

In [2]:
generate_bigrams(['This', 'film', 'is', 'terrible'])

['This', 'film', 'is', 'terrible', 'film is', 'This film', 'is terrible']

Con TorchText nei Fields abbiamo un argomento preprocessing. Una funzione può essere applicata alla frase dopo che è stata trasformata in una lista di token, ma prima che i token siano stati trasformati in numeri. E' questo il punto in cui metteremo la nostra funzione generate_bigrams.

Non stiamo utilizzando una RNN e dunque non serve utilizzare una gestione delle packed padded sequences, dunque non serve impostare include_lengths = True.

In [3]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', preprocessing = generate_bigrams)
LABEL = data.LabelField(dtype = torch.float)

Come prima, carichiamo il dataset e lo spezziamo in tre

In [4]:
import random

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

Creiamo il vocabolario e cariciamo i vettori di word embedding pre-trained.

In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

Creiamo gli iteratori per il dataset

In [6]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

Costruiamo il modello
----
Questo modello ha molti meno parametri dei precedenti, ha solo 2 layer con parametri, l'embedding layer e il linear layer.
Prima viene calcolato il word embedding per ogni parola usando l'Embedding layer (blu), poi viene calcolato la media di tutte le parole del word embedding (rosa) e il tutto viene passato al Linear layer (grigio)

<img src="./images/fasttext.png">

Implementiamo la media con la funzione avg_pool2d (average pool 2-dimensions). Inizialmente, possiamo pensare che usare un pooling a 2 dimensioni sis strano, la nostra frase è a una dimensione non a 2. Comunque possiamo pensare come una word embeddings a 2 dimensioni con le parole lungo un asse e l'embedding nell'altra.

Nella immagine sotto abbiamo un esempio di pooling con un vettore di embedding di lunghezza 5, le parole sono lungo l'asse verticale con l'embedding lungo l'asse orizzontale.  

<img src="./images/pooling.png">

alla fine di questa operazione otterremo un tensore [1 x 5] e questo tensore sarà letto dalla rete neuronale

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

instanziamo il modello come sempre fatto sino ad ora

In [8]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

Andiamo a vedere i parametri del modello e vediamo che sono la metà del modello precedente

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


Ora copio i vettori nel nostro strato di embedding.

In [10]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.2463,  0.5749, -0.3656,  ...,  0.4434,  0.5794, -0.3972],
        [ 0.9874, -0.3359,  1.8736,  ..., -0.2993, -0.3686,  0.9624],
        [ 0.4455,  0.1342, -2.3334,  ...,  0.9246,  0.3400, -1.0022]])

Train del modello
----

Andiamo a fare esattamente le stesse cose del notebook precedente

In [11]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

Creiamo ora una funzione per il train del modello 

**Nota:** non stiamo usando il dropout anche se sarebbe consigliato

In [12]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

creiamo la nostra funzione di eval

In [13]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Come prima mi creo le mie funzioni per misurare il tempo e lancio il train

In [14]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [15]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 9s
	Train Loss: 0.692 | Train Acc: 51.62%
	 Val. Loss: 0.674 |  Val. Acc: 62.46%
Epoch: 02 | Epoch Time: 0m 7s
	Train Loss: 0.684 | Train Acc: 57.73%
	 Val. Loss: 0.644 |  Val. Acc: 70.48%
Epoch: 03 | Epoch Time: 0m 7s
	Train Loss: 0.670 | Train Acc: 66.43%
	 Val. Loss: 0.593 |  Val. Acc: 72.41%
Epoch: 04 | Epoch Time: 0m 7s
	Train Loss: 0.646 | Train Acc: 70.88%
	 Val. Loss: 0.543 |  Val. Acc: 74.29%
Epoch: 05 | Epoch Time: 0m 7s
	Train Loss: 0.614 | Train Acc: 75.14%
	 Val. Loss: 0.500 |  Val. Acc: 76.53%
Epoch: 06 | Epoch Time: 0m 7s
	Train Loss: 0.579 | Train Acc: 78.27%
	 Val. Loss: 0.461 |  Val. Acc: 78.61%
Epoch: 07 | Epoch Time: 0m 7s
	Train Loss: 0.542 | Train Acc: 81.38%
	 Val. Loss: 0.439 |  Val. Acc: 80.49%
Epoch: 08 | Epoch Time: 0m 7s
	Train Loss: 0.505 | Train Acc: 82.99%
	 Val. Loss: 0.418 |  Val. Acc: 82.00%
Epoch: 09 | Epoch Time: 0m 7s
	Train Loss: 0.472 | Train Acc: 85.13%
	 Val. Loss: 0.402 |  Val. Acc: 83.32%
Epoch: 10 | Epoch Time: 0m 7

Aniamo a vedere il risultato sul dataset di test.

In [16]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.399 | Test Acc: 84.18%


Abbiamo ottenuto un buon risultato e un sistema con ottime performance nel tempo.

Input personalizzato
----

Come prima andiamo ad eseguire un test su una nostra frase personalizzata. 

In [17]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

un esempio di recensione negativa

In [22]:
predict_sentiment(model, "This film wasn't great")

0.026226362213492393

un esempio di recensione positiva

In [23]:
predict_sentiment(model, "This film was great")

1.0